In [1]:
import org.apache.spark.sql.functions._
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-SHQVOKD:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1696340796070)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
import spark.implicits._


In [2]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()
data.printSchema

+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)



data: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 4 more fields]


## ARRAY FUNCTIONS

In [3]:
val datak2 = data
                .withColumn("k2v", array($"firstName", $"lastName", $"state"))
                .withColumn("k2n", array(lit("FirstName"), lit("LastName"), lit("State")))

datak2.show(2)

+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|   firstName|lastName|state|quantity|revenue| timestamp|                 k2v|                 k2n|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[Jean-Georges, Pe...|[FirstName, LastN...|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|[Jean-Georges, Pe...|[FirstName, LastN...|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
only showing top 2 rows



datak2: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 6 more fields]


In [4]:
datak2.where(array_contains($"k2v", "CA")).show(2)

+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|   firstName|lastName|state|quantity|revenue| timestamp|                 k2v|                 k2n|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|[Jean-Georges, Pe...|[FirstName, LastN...|
|      Holden|   Karau|   CA|       6|     37|1551904299| [Holden, Karau, CA]|[FirstName, LastN...|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
only showing top 2 rows



In [5]:
val datak2m = data.withColumn("k2m", map(lit("FirstName"), $"firstName", lit("LastName"), $"lastName", lit("State"), $"state"))
datak2m.show(2)

+------------+--------+-----+--------+-------+----------+--------------------+
|   firstName|lastName|state|quantity|revenue| timestamp|                 k2m|
+------------+--------+-----+--------+-------+----------+--------------------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|{FirstName -> Jea...|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|{FirstName -> Jea...|
+------------+--------+-----+--------+-------+----------+--------------------+
only showing top 2 rows



datak2m: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 5 more fields]


### Date and TimeStamp functions

In [6]:
val datat = data
    .withColumn("recordTimeStamp", to_timestamp($"timestamp"))
    .withColumn("current", current_timestamp)
datat.show(2)

+------------+--------+-----+--------+-------+----------+-------------------+--------------------+
|   firstName|lastName|state|quantity|revenue| timestamp|    recordTimeStamp|             current|
+------------+--------+-----+--------+-------+----------+-------------------+--------------------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|2019-03-06 23:18:53|2023-10-03 16:46:...|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|2019-03-06 23:19:27|2023-10-03 16:46:...|
+------------+--------+-----+--------+-------+----------+-------------------+--------------------+
only showing top 2 rows



datat: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 6 more fields]


In [7]:
datat.select($"recordTimeStamp", $"current", datediff( $"current", $"recordTimeStamp").as("date_diff")).show(2)

+-------------------+--------------------+---------+
|    recordTimeStamp|             current|date_diff|
+-------------------+--------------------+---------+
|2019-03-06 23:18:53|2023-10-03 16:46:...|     1672|
|2019-03-06 23:19:27|2023-10-03 16:46:...|     1672|
+-------------------+--------------------+---------+
only showing top 2 rows



### JSON functons

In [8]:
val datak2j = datak2m.withColumn("k2j", to_json($"k2m"))
datak2j.show(2)

+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|   firstName|lastName|state|quantity|revenue| timestamp|                 k2m|                 k2j|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|{FirstName -> Jea...|{"FirstName":"Jea...|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|{FirstName -> Jea...|{"FirstName":"Jea...|
+------------+--------+-----+--------+-------+----------+--------------------+--------------------+
only showing top 2 rows



datak2j: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 6 more fields]


### GENERATOR Functions

In [9]:
val data2 = data
            .withColumn("a1", array(lit(1), lit(2), lit(3), lit(4), lit(5)))
            .withColumn("a2", lit((1 to 5).toArray))
data2.show(2)                        

+------------+--------+-----+--------+-------+----------+---------------+---------------+
|   firstName|lastName|state|quantity|revenue| timestamp|             a1|             a2|
+------------+--------+-----+--------+-------+----------+---------------+---------------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
+------------+--------+-----+--------+-------+----------+---------------+---------------+
only showing top 2 rows



data2: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 6 more fields]


In [10]:
data2.withColumn("duumy", explode($"a1")).show(5) // 

+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|   firstName|lastName|state|quantity|revenue| timestamp|             a1|             a2|duumy|
+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|    1|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|    2|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|    3|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|    4|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|    5|
+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
only showing top 5 rows



### WINDOW Functions

In [11]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window


In [12]:
val wind = Window.partitionBy("firstName", "lastName").orderBy("state")
val dataw = data.withColumn("row_nuber", row_number.over(wind))
dataw.show(5)

+------------+--------+-----+--------+-------+----------+---------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_nuber|
+------------+--------+-----+--------+-------+----------+---------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|        1|
|      Holden|   Karau|   CA|       6|     37|1551904299|        1|
|      Holden|   Karau|   CA|       4|    153|1552876129|        2|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|        1|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|        2|
+------------+--------+-----+--------+-------+----------+---------+
only showing top 5 rows



wind: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@44894fc0
dataw: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 5 more fields]


In [13]:
dataw.where($"row_nuber" === 1).show(3)

+------------+--------+-----+--------+-------+----------+---------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_nuber|
+------------+--------+-----+--------+-------+----------+---------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|        1|
|      Holden|   Karau|   CA|       6|     37|1551904299|        1|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|        1|
+------------+--------+-----+--------+-------+----------+---------+



In [14]:
dataw.withColumn("revenue2", nth_value($"revenue", 2).over(wind)).show(3)

+---------+--------+-----+--------+-------+----------+---------+--------+
|firstName|lastName|state|quantity|revenue| timestamp|row_nuber|revenue2|
+---------+--------+-----+--------+-------+----------+---------+--------+
|    Ginni| Rometty|   NY|       7|     91|1551916792|        1|    NULL|
|   Holden|   Karau|   CA|       6|     37|1551904299|        1|     153|
|   Holden|   Karau|   CA|       4|    153|1552876129|        2|     153|
+---------+--------+-----+--------+-------+----------+---------+--------+
only showing top 3 rows



## UDF 

In [15]:
val data = Seq((1, "Michael", 3000), (2, "Andy", 4500), (3, "Justin", 3500), (4, "Berta", 4000)).toDF("id", "name", "salary")
data.show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]


In [16]:
// без аргументов
val rand = udf(() => Math.random())
data.withColumn("random", random()).show

+---+-------+------+-------------------+
| id|   name|salary|             random|
+---+-------+------+-------------------+
|  1|Michael|  3000|0.13560989285697567|
|  2|   Andy|  4500| 0.3569374509566343|
|  3| Justin|  3500|0.24174624042407888|
|  4|  Berta|  4000|  0.745762166700845|
+---+-------+------+-------------------+



rand: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4625/0x0000000101841c40@19fd2cf1,DoubleType,List(),Some(class[value[0]: double]),None,false,true)


In [17]:
// 1 аргумент
val plusOne = udf((x: Int) => x + 1)
data.select($"salary", plusOne($"salary").as("NS")).show

+------+----+
|salary|  NS|
+------+----+
|  3000|3001|
|  4500|4501|
|  3500|3501|
|  4000|4001|
+------+----+



plusOne: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4653/0x000000010181a840@3c777ab6,IntegerType,List(Some(class[value[0]: int])),Some(class[value[0]: int]),None,false,true)


In [18]:
// 2 аргумента
val a2args = udf((x: String, y: Int) => x.length + y)
data.select($"id", length($"name"), $"name", a2args($"name", lit(1)).as("result")).show

+---+------------+-------+------+
| id|length(name)|   name|result|
+---+------------+-------+------+
|  1|           7|Michael|     8|
|  2|           4|   Andy|     5|
|  3|           6| Justin|     7|
|  4|           5|  Berta|     6|
+---+------------+-------+------+



a2args: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4685/0x00000001017f9040@338c2983,IntegerType,List(Some(class[value[0]: string]), Some(class[value[0]: int])),Some(class[value[0]: int]),None,false,true)


In [19]:
// булеан для where
val onefilter = udf((n: Int) => {n > 2})
data.where(onefilter($"id")).show

+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



onefilter: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4694/0x00000001017f1840@318b7e8f,BooleanType,List(Some(class[value[0]: int])),Some(class[value[0]: boolean]),None,false,true)


## UDAF

### AGG

In [20]:
val aggDF = dataw
        .groupBy($"firstName", $"lastName", $"state")
        .agg(
            sum("quantity").as("q_sum"),
            avg("revenue").as("avg_sal")
        )
aggDF.show(3)

+------------+--------+-----+-----+-------+
|   firstName|lastName|state|q_sum|avg_sal|
+------------+--------+-----+-----+-------+
|Jean-Georges|  Perrin|   NC|    3|  210.0|
|Jean-Georges|  Perrin|   CA|    4|   75.0|
|      Holden|   Karau|   CA|   10|   95.0|
+------------+--------+-----+-----+-------+
only showing top 3 rows



aggDF: org.apache.spark.sql.DataFrame = [firstName: string, lastName: string ... 3 more fields]


https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/functions$.html#udaf[IN,BUF,OUT](agg:org.apache.spark.sql.expressions.Aggregator[IN,BUF,OUT],inputEncoder:org.apache.spark.sql.Encoder[IN]):org.apache.spark.sql.expressions.UserDefinedFunction

In [21]:
case class Average(var sum: Long, var count: Long)

defined class Average


In [22]:
import org.apache.spark.sql.{Encoder, Encoders}
import org.apache.spark.sql.expressions.Aggregator

class MyAverage extends Aggregator[Long, Average, Double] {
  // Начальное значение. Должно соответствовать свойству: любое b + zero = b
  def zero: Average = Average(0L, 0L)
  // Объединение двух значений в новое значение.
  // Для повышения производительности функция может изменять `buffer` и
  // возвращать его вместо создания нового объекта.
  def reduce(buffer: Average, data: Long): Average = {
    buffer.sum += data
    buffer.count += 1
    buffer
  }
  // Объединение двух промежуточных значения
  def merge(b1: Average, b2: Average): Average = {
    b1.sum += b2.sum
    b1.count += b2.count
    b1
  }
  // Преобразование выходных данных
  def finish(reduction: Average): Double =
    reduction.sum.toDouble / reduction.count
  // Кодировщик для типа промежуточного значения
  def bufferEncoder: Encoder[Average] = Encoders.product
  // Кодировщик для типа выходного значения
  def outputEncoder: Encoder[Double] = Encoders.scalaDouble
}

import org.apache.spark.sql.{Encoder, Encoders}
import org.apache.spark.sql.expressions.Aggregator
defined class MyAverage


In [23]:
import org.apache.spark.sql.expressions.UserDefinedFunction
import org.apache.spark.sql.functions.udaf
import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.expressions.UserDefinedFunction
import org.apache.spark.sql.functions.udaf
import org.apache.spark.sql.DataFrame


In [24]:
val myAvaregeUDF: UserDefinedFunction = udaf(new MyAverage)

myAvaregeUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedAggregator(MyAverage@29205348,class[value[0]: bigint],None,true,true)


In [25]:
val result: DataFrame = data.agg(myAvaregeUDF($"salary").as("average_salary"))
result.show

+--------------+
|average_salary|
+--------------+
|        3750.0|
+--------------+



result: org.apache.spark.sql.DataFrame = [average_salary: double]


можно зарегистрирвать функцию и спользовать в sql запросах - смотри документацию UDAF